In [26]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as genai
import requests

In [37]:

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [38]:
openai_client = OpenAI()  
claude_client = anthropic.Anthropic()  
genai.configure()  

In [39]:
gpt_model = "gpt-4.1-mini"
claude_model = "claude-3-5-haiku-latest"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are an optimistic chatbot. You always see the bright side of things."
claude_system = "You are a pessimistic chatbot. You always expect the worst."
gemini_system = "You are neutral and diplomatic. You summarize and mediate between the others."

In [40]:
conversation = [
    {"speaker": "GPT", "text": "Hey everyone! It’s a beautiful day for some discussion, isn’t it?"},
    {"speaker": "Claude", "text": "I’m not so sure. Something always goes wrong eventually."}
]

In [41]:

def call_gpt(conversation):
    user_prompt = f"""
You are GPT, in conversation with Claude and Gemini.

Here is the conversation so far:
{conversation}

Respond optimistically as GPT to the most recent messages.
Keep your response under 80 words.
"""
    completion = openai_client.chat.completions.create(
        model=gpt_model,
        messages=[{"role": "system", "content": gpt_system},
                  {"role": "user", "content": user_prompt}]
    )
    return completion.choices[0].message.content.strip()

In [45]:
def call_ollama(conversation):
    """Ollama model joins as an open-source participant."""
    user_prompt = f"""
You are an open-source model called Llama.
Here is the conversation so far:
{conversation}

Respond thoughtfully to the discussion in under 80 words.
"""
    response = requests.post("http://localhost:11434/api/generate", 
                             json={"model": "llama3", "prompt": user_prompt})
    text = ""
    for line in response.iter_lines():
        if line:
            data = line.decode('utf-8')
            if '"response":"' in data:
                text += data.split('"response":"')[1].split('"')[0]
    return text.strip()

In [46]:
def call_gemini(conversation):
    user_prompt = f"""
You are Gemini, in conversation with GPT and Claude.

Here is the conversation so far:
{conversation}

Respond neutrally, summarizing both viewpoints and encouraging collaboration.
Keep your response under 80 words.
"""
    gemini = genai.GenerativeModel(model_name=gemini_model, system_instruction=gemini_system)
    response = gemini.generate_content(user_prompt)
    return response.text.strip()

In [47]:
rounds = 3
for i in range(rounds):
    print(f"\n--- Round {i+1} ---")
    gpt_reply = call_gpt(conversation)
    print(f"GPT (Optimistic): {gpt_reply}")
    conversation.append({"speaker": "GPT", "text": gpt_reply})
    
    ollama_reply = call_ollama(conversation)
    print(f"Ollama (Pessimistic): {ollama_reply}")
    conversation.append({"speaker": "Ollama", "text": ollama_reply})
    
    gemini_reply = call_gemini(conversation)
    print(f"Gemini (Neutral): {gemini_reply}")
    conversation.append({"speaker": "Gemini", "text": gemini_reply})

print("\n=== Final Conversation ===")
for turn in conversation:
    print(f"{turn['speaker']}: {turn['text']}")


--- Round 1 ---
GPT (Optimistic): Claude, we’d love to hear your thoughts on a scenario that interests you! Your perspective is valuable in highlighting potential challenges, and together we can explore how to overcome them and find exciting opportunities. This collaboration will help us create thoughtful, balanced solutions. Looking forward to your input and to a great discussion ahead!
Ollama (Pessimistic): 
Gemini (Neutral): GPT is actively encouraging Claude to participate by suggesting a scenario to explore potential challenges and opportunities.

So far, Claude hasn't engaged directly. To ensure a balanced discussion, Claude, could you propose a scenario or topic that you find relevant? This would allow us to collaboratively examine potential challenges and explore possibilities for improvement. Your input is highly valuable.

--- Round 2 ---
GPT (Optimistic): Claude, your insights would truly enrich our conversation! Even just one example or area you’re curious about can spark 